# Distributions

## Libraries

In [ ]:
import os
import sys
import importlib
from abc import abstractmethod
from typing import Tuple, Any

import numpy as np
import torch
import tensorflow as tf
import matplotlib.pyplot as plt

## Functions

In [ ]:
def compare_tensors(a, b, rtol=1e-1, atol=1e-1, name=""):
    are_close = np.allclose(a, b, rtol=rtol, atol=atol)
    max_diff = np.max(np.abs(a - b))
    print(f"{name} are close: {are_close}")
    print(f"Max absolute difference for {name}: {max_diff}")
    return are_close

## Random seeds

Set random seeds for reproducibility

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
tf.random.set_seed(0)

## Illia

When the backend is selected we can import illia, if we want to change the backend we need to restart the kernel. The backend can't be changed dynamically.

In [ ]:
import illia

We can check the available backends using the following function:

In [ ]:
illia.show_available_backends()

Test params and utils:

In [ ]:
shape = (3, 2)
mu_prior = 0.0
std_prior = 0.1
mu_init = 0.0
rho_init = -7.0

Initialize class distributions:

In [ ]:
from illia.torch.distributions.dynamic.gaussian import GaussianDistribution as TorchGaussianDistribution
from illia.tf.distributions.dynamic.gaussian import GaussianDistribution  as TFGaussianDistribution

In [ ]:
torch_dynamic_dist = TorchGaussianDistribution(
    shape=shape, mu_init=mu_init, rho_init=rho_init
)

In [ ]:
tf_dynamic_dist = TFGaussianDistribution(
    shape=shape, mu_init=mu_init, rho_init=rho_init
)

Test 1 - Distributions sampling

In [ ]:
print("Test 1: Sampling")

n_samples = 10000
torch_samples = np.array([torch_dynamic_dist.sample().detach().cpu().numpy() for _ in range(n_samples)])
tf_samples = np.array([tf_dynamic_dist.sample().numpy() for _ in range(n_samples)])

Compare means

In [ ]:
torch_mean = np.mean(torch_samples, axis=0)
tf_mean = np.mean(tf_samples, axis=0)
compare_tensors(torch_mean, tf_mean, name="Means")

Compare standard deviations

In [ ]:
torch_std = np.std(torch_samples, axis=0)
tf_std = np.std(tf_samples, axis=0)
compare_tensors(torch_std, tf_std, name="Standard deviations")

Test 2 - Distributions log probs

In [ ]:
print("\nTest 2: Log probability")

x = np.random.randn(*shape).astype(np.float32)
torch_log_prob = torch_dynamic_dist.log_prob(torch.tensor(x, dtype=torch.float32)).detach().cpu().numpy()
tf_log_prob = tf_dynamic_dist.log_prob(tf.constant(x, dtype=tf.float32)).numpy()
compare_tensors(torch_log_prob, tf_log_prob, rtol=1e-1, atol=1e-1, name="Log probabilities")

Test 3 - Distributions num of params

In [ ]:
print("\nTest 3: Number of parameters")
print("PyTorch num params:", torch_dynamic_dist.num_params)
print("TensorFlow num params:", tf_dynamic_dist.num_params)
print("Num params are equal:", torch_dynamic_dist.num_params == tf_dynamic_dist.num_params)